<a href="https://colab.research.google.com/github/rainson/colab_nbs/blob/main/ytd2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Basic functions and imports


In [ ]:
!pip install pytube
!pip install youtube_transcript_api
!pip install vtt_to_srt3

In [2]:
from pytube import YouTube, Playlist
from youtube_transcript_api import YouTubeTranscriptApi
import json
import requests
from lxml import html
from vtt_to_srt import vtt_to_srt

# from vtt_to_srt import vtts_to_srt
from youtube_transcript_api.formatters import JSONFormatter
from youtube_transcript_api.formatters import TextFormatter
from youtube_transcript_api.formatters import WebVTTFormatter

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

def download(idx,yturl,out_dir):
  formatter = WebVTTFormatter()
  yt = YouTube(yturl)
  yt.streams.filter(file_extension="mp4")
  title = yt.title
  title="{}".format(idx)+"".join([s.capitalize() for s in title.split(" ")])
  print("start {}".format(title))
  try:
    yt.streams.get_by_itag(22).download(output_path=out_dir,filename=title)
  except Exception:
    yt.streams.get_by_itag(18).download(output_path=out_dir,filename=title)

  try:
    caption = yt.captions["en"]
    caption.download(title=title,output_path=out_dir)
  except Exception as ex:
    print(f"no en subtitle in {yt.title} finished")
  try:
    caption = yt.captions["a.en"]
    caption.download(title=title,output_path=out_dir)
  except Exception as ex:
    print(f"no a.en subtitle in {yt.title} finished")
  

  try:
    transcript_list = YouTubeTranscriptApi.list_transcripts(yt.video_id)
    transcript_list

    transcript = transcript_list.find_transcript(['en'])
    translated_transcript = transcript.translate('zh-Hans')
    zh_trans = translated_transcript.fetch()
    en_trans = transcript.fetch()

    zh_trans = formatter.format_transcript(zh_trans)
    en_trans=formatter.format_transcript(en_trans)

    fname = out_dir+"/{}(en).vtt".format(title)
    with open(fname, "w") as fh:
        fh.write(en_trans)
    # vtt_to_srt(fname)
    fname = out_dir+"/{}(zh).vtt".format(title)
    with open(fname, "w") as fh:
        fh.write(zh_trans)
    # vtt_to_srt(fname)
    fname =  out_dir+"/{}(zhen).vtt".format(title)
    with open(fname, "w") as fh:
        fh.write(zh_trans+en_trans)
    # vtt_to_srt(fname)
  except Exception:
    pass
  print("finish {}".format(title))

# Find urls in one page


In [ ]:
import re
# page_url = "https://www.youtube.com/channel/UCAKQnU09rjmdVRb3iAogCiQ/videos" #Atomic scale Jose Lado
page_url = "https://www.youtube.com/c/ILSIITR/videos" #MOU Institute Lecture Series
page = requests.get(page_url)
webpage = html.fromstring(page.content)

video_ids = re.findall(r"watch\?v=(\S{11})", page.content.decode())
print(video_ids)
urls = ["https://www.youtube.com/watch?v={}".format(vid) for vid in video_ids]

['tEdFbAYjDtU', 'KsIKadfmhpU', 'nVRwyr2O3gU', '4XXeJ-BWt80', 'e7Z-eup1f3Q', 'JniVQTuZR3c', 'HBBkOqHGcjs', 'LE5-NibKJ9s', 'OFpzfokko0M', '4gh3SZJajyY', 'DaRwdfMiR3w', 'TwayPGEXx1E', 'C1omCSbEHq0', 'WtoNiMk8t2E', 'DOHtdSeiWco', 'jmB5vIadLi8', 'gHy14szhi0o', 'RYKyshYzXd8', 'Q3qDjcVvtA8', 'CB_T8DRIIS8', 'GGISr6FsD70', '-zuzOmM-r3Q', 'RM4yPpErVv4', 'kaZ34JyvS-8', 'gkPMyUK9csI', 'fl6cEAN0Xp8', 'CeIV8NA9m78', 'ot8VjDsxRFM', 'XwSvpb_j-KM', '8-XW7B6y2lk']


# Urls in playlist


In [6]:
# plisturl = "https://www.youtube.com/playlist?list=PL7OOhDqj9FBuK3han8ipaKnz2SaGUjvGx"# topo cm
# plisturl="https://www.youtube.com/playlist?list=PLygH4pJ_4EhrwydaPvccVQYt1W6ytaYbU" # tms19
# plisturl = "https://www.youtube.com/playlist?list=PLygH4pJ_4EhogZda-GSFOBhlr2lSHn_p2" #tms18
# plisturl = "https://www.youtube.com/playlist?list=PLVqoFIN-64LnUQxbX2FeIGYQgX_VTWm-m" #tms17
# plisturl = "https://www.youtube.com/playlist?list=PLLv6hTkOwhEl1itfLvR-IcvOFt-aCxSzy" tms16
# plisturl = "https://www.youtube.com/playlist?list=PLE9Qrf4CJnRHxa_OwN7xc2bXacnWzCy1x" #opensys1
# plisturl = "https://www.youtube.com/playlist?list=PLd9T7y2ABtzJuN2YgujzWZ6fL-llEyuUd" #opensys2
# plisturl = "https://www.youtube.com/watch?v=g2RwzmG0spo&list=PL2IOBr0XELQSaB14lQgdFNvKHzs0W15hh" #UTphysics2020
# plisturl = "https://www.youtube.com/watch?v=BSV8zKbKc_Y&list=PL2IOBr0XELQQUjDBrh2tvSHhjHu9PHFYF"
# plisturl = "https://www.youtube.com/playlist?list=PLZee5Mirzz8ULGEdfuWM0DaoM-Yqtz9r8" #pcmp2014
# plisturl = "https://www.youtube.com/playlist?list=PLZee5Mirzz8WkK2EtlY1De7msJ6x9vP3T" #pcmp2015
# plisturl = "https://www.youtube.com/playlist?list=PLZee5Mirzz8UaZUDJZmh1BCV3xIjEPNEw" #pcmp2016
# plisturl = "https://www.youtube.com/playlist?list=PL04QVxpjcnjgmUoD6tlO_0YT1k5GYgQWi" #flucations in non equlibrium 
plisturl = "https://www.youtube.com/playlist?list=PLgLljLUllyOcO6g2hoVlPiIKYYdpnlrwP" #IQUIST Seminars need reversed 更新中
plist = Playlist(plisturl)
urls = plist.video_urls
urls = urls[::-1] # reversed for some special case
print(len(urls))

30


# Url manully


In [ ]:
## non hermitian 2021
# urls = ["https://www.youtube.com/watch?v=MiyTEVq8BJ0",
#         "https://www.youtube.com/watch?v=wor5vdmBH_E",
#         "https://www.youtube.com/watch?v=s3qXZbU7ivk",
#         "https://www.youtube.com/watch?v=ON8hMzgLEHo",
#         "https://www.youtube.com/watch?v=gtC1LBLYrPk",
#         "https://www.youtube.com/watch?v=b1qrdafHMp0",
#         "https://www.youtube.com/watch?v=xcIFxekzy3Q",
#         "https://www.youtube.com/watch?v=BnIuitcOhmE",
#         "https://www.youtube.com/watch?v=1zV49ovoGN8",
#         "https://www.youtube.com/watch?v=Tpm0rSPw_9I"
#         ] 

# Download task

In [ ]:
# urls = ["https://www.youtube.com/watch?v=MiyTEVq8BJ0","https://www.youtube.com/watch?v=wor5vdmBH_E","https://www.youtube.com/watch?v=s3qXZbU7ivk","https://www.youtube.com/watch?v=ON8hMzgLEHo"]
# urls = ["https://www.youtube.com/watch?v=MiyTEVq8BJ0"]
out_dir = "/content/drive/Shareddrives/OBA/tms/tms17"

for idx, yturl in enumerate(urls): #plist.video_urls
  try:
    download(idx+1,yturl, out_dir)
  except Exception as ex:
    print(f"exception {ex}")
vtt_to_srt.vtts_to_srt(out_dir, rec = True)

no en subtitle in TMS17. Leslie Schoop. Prediction, growth, and characterization of topological materials (I) finished
finish 16Tms17.LeslieSchoop.Prediction,Growth,AndCharacterizationOfTopologicalMaterials(i)
start 17Tms17.LeslieSchoop.Prediction,Growth,AndCharacterizationOfTopologicalMaterials(i)i
no en subtitle in TMS17. Leslie Schoop. Prediction, growth, and characterization of topological materials (I)I finished
finish 17Tms17.LeslieSchoop.Prediction,Growth,AndCharacterizationOfTopologicalMaterials(i)i
start 18Tms17.ClaudiaFelser.SearchOfNewTopologicalMatter(i)
no en subtitle in TMS17. Claudia Felser. Search of new topological matter (I) finished
no a.en subtitle in TMS17. Claudia Felser. Search of new topological matter (I) finished
finish 18Tms17.ClaudiaFelser.SearchOfNewTopologicalMatter(i)
start 19Tms17.ClaudiaFelser.SearchOfNewTopologicalMatter(ii)
no en subtitle in TMS17. Claudia Felser. Search of new topological matter (II) finished
no a.en subtitle in TMS17. Claudia Felser

In [ ]:
!cd /content/drive/Shareddrives/OBA/tms; tar -cvf /content/drive/Shareddrives/OBA/tms.tar ./*
